# MaterialPointSolver.jl Showcase 04

Author: Zenan Huo <br>
Date: 25-Aug-2024 <br>

---

This is the case for wish card 2024. JUST FOR FUN :)

In [39]:
using MaterialPointSolver
using KernelAbstractions
using CUDA

In [40]:
warmup(Val(:CUDA)) # optional

[ Info: code warm-up, wait a moment 🔥


In [41]:
init_grid_space_x = 0.1
init_grid_space_y = 0.1
init_grid_space_z = 0.1
init_grid_range_x = [-5.3,  5.3]
init_grid_range_y = [-5.3,  5.3]
init_grid_range_z = [-0.3, 23.3]
init_mp_in_space  = 2
init_project_name = "2024_wish_card"
init_project_path = joinpath(@__DIR__, "outputs", init_project_name)
init_constitutive = :druckerprager
init_gravity      = -9.8
init_ζs           = 0
init_ρs           = 2600
init_ν            = 0.3
init_E            = 1e7
init_Ks           = init_E / (3 * (1 - 2 * init_ν))
init_G            = init_E / (2 * (1 +     init_ν))
init_T            = 18
init_Te           = 0
init_ΔT           = 0.1 * init_grid_space_x / sqrt((init_Ks + 4/3 * init_G) / init_ρs)
init_step         = (t = floor(init_T / init_ΔT / 150); t<10 ? 1 : t)
init_σt           = 0
init_ϕ            = deg2rad(35)
init_c            = 0
init_ψ            = 0
init_basis        = :uGIMP
init_phase        = 1
init_NIC          = 64
iInt              = Int64
iFloat            = Float64;

### Parameters Setup

In [42]:
args = Args3D{iInt, iFloat}(
    Ttol         = init_T,
    ΔT           = init_ΔT,
    Te           = init_Te,
    time_step    = :fixed,
    FLIP         = 1,
    PIC          = 0,
    ζs           = init_ζs,
    gravity      = init_gravity,
    project_name = init_project_name,
    project_path = init_project_path,
    constitutive = init_constitutive,
    animation    = true,
    hdf5         = false,
    hdf5_step    = init_step,
    MVL          = true,
    device       = :CUDA,
    coupling     = :OS,
    progressbar  = true,
    basis        = init_basis
)

┌ Warning: HDF5 forced ON due to the animation
└ @ MaterialPointSolver ~/Workbench/MaterialPointSolver.jl/src/types/modelargs.jl:163


Args3D{Int64, Float64}
──────────────────────
project name    : 2024_wish_card
project path    : /home/zhuo/Workbench/MaterialPointSolver.jl/examples/outputs/2024_wish_card
precision       : FP64
constitutive    : druckerprager
basis method    : uGIMP
mitigate vollock: true
coupling scheme : OS


### Background Grid Setup

In [43]:
grid = Grid3D{iInt, iFloat}(
    NIC      = init_NIC,
    range_x1 = init_grid_range_x[1],
    range_x2 = init_grid_range_x[2],
    range_y1 = init_grid_range_y[1],
    range_y2 = init_grid_range_y[2],
    range_z1 = init_grid_range_z[1],
    range_z2 = init_grid_range_z[2],
    space_x  = init_grid_space_x,
    space_y  = init_grid_space_y,
    space_z  = init_grid_space_z,
    phase    = init_phase
)

Grid3D{Int64, Float64}
──────────────────────
node: 2713413
cell: 2651696


### Material Points Setup

In [44]:
space_x = grid.space_x / init_mp_in_space
space_y = grid.space_y / init_mp_in_space
space_z = grid.space_z / init_mp_in_space
x_tmp, y_tmp, z_tmp = meshbuilder( -2.5 + space_x / 2 : space_x :  2.5 - space_x / 2,
                                   -2.5 + space_y / 2 : space_y :  2.5 - space_y / 2,
                                   17   + space_z / 2 : space_z : 22   - space_z / 2)
mp_num = length(x_tmp)
mp_ρs  = ones(mp_num).*init_ρs
mp     = Particle3D{iInt, iFloat}(space_x=space_x, space_y=space_y, space_z=space_z,
    pos=[x_tmp y_tmp z_tmp], ρs=mp_ρs, NIC=init_NIC, phase=init_phase)

Particle3D{Int64, Float64}
──────────────────────────
particle: 1000000


### Particle Property Setup

In [45]:
mp_layer   = ones(mp_num)
mp_ν       = [init_ν]
mp_E       = [init_E]
mp_G       = [init_G]
mp_σt      = [init_σt]
mp_ϕ       = [init_ϕ]
mp_c       = [init_c]
mp_ψ       = [init_ψ]
mp_Ks      = [init_Ks]
pts_attr   = ParticleProperty{iInt, iFloat}(layer=mp_layer, ν=mp_ν, E=mp_E, G=mp_G, 
    σt=mp_σt, ϕ=mp_ϕ, c=mp_c, ψ=mp_ψ, Ks=mp_Ks)

ParticleProperty{Int64, Float64}
────────────────────────────────
material partition: 1


### Boundary Condition Nodes Index

In [46]:
vx_idx  = zeros(iInt, grid.node_num)
vy_idx  = zeros(iInt, grid.node_num)
vz_idx  = zeros(iInt, grid.node_num)
tmp_idx = findall(i->((grid.pos[i, 1] < -2.5 || grid.pos[i, 1] > 2.5) && (23 > grid.pos[i, 3] > 16)) ||
                     ( grid.pos[i, 1] < -5   || grid.pos[i, 1] > 5  ) || 
                     ( grid.pos[i, 3] <  0   || grid.pos[i, 3] > 23 ), 1:grid.node_num)
tmp_idy = findall(i->((grid.pos[i, 2] < -2.5 || grid.pos[i, 2] > 2.5) && (23 > grid.pos[i, 3] > 16)) ||
                     ( grid.pos[i, 2] < -5   || grid.pos[i, 2] > 5  ) ||
                     ( grid.pos[i, 3] <  0   || grid.pos[i, 3] > 23 ), 1:grid.node_num)
tmp_idz = findall(i->((grid.pos[i, 2]^2 > 0.8^2 - grid.pos[i, 1]^2) && (16.5 ≤ grid.pos[i, 3] ≤ 17)) ||
                     ( grid.pos[i, 3] ≤  0   || grid.pos[i, 3] > 23 ), 1:grid.node_num)
vx_idx[tmp_idx] .= 1
vy_idx[tmp_idy] .= 1
vz_idx[tmp_idz] .= 1
bc = VBoundary3D{iInt, iFloat}(
    Vx_s_Idx = vx_idx,
    Vx_s_Val = zeros(grid.node_num),
    Vy_s_Idx = vy_idx,
    Vy_s_Val = zeros(grid.node_num),
    Vz_s_Idx = vz_idx,
    Vz_s_Val = zeros(grid.node_num)
)

VBoundary3D{Int64, Float64}
───────────────────────────
velocity boundary


### MPM Solver

In [47]:
@kernel inbounds=true function testG2P_OS!(
    grid::    KernelGrid3D{T1, T2},
    mp  ::KernelParticle3D{T1, T2}
) where {T1, T2}
    ix = @index(Global)
    if ix <= mp.num
        dF1 = dF2 = dF3 = dF4 = dF5 = dF6 = dF7 = dF8 = dF9 = T2(0.0)
        for iy in Int32(1):Int32(mp.NIC)
            if mp.Ni[ix, iy] != T2(0.0)
                p2n = mp.p2n[ix, iy]
                ∂Nx = mp.∂Nx[ix, iy]; ds1 = grid.Δd_s[p2n, 1]
                ∂Ny = mp.∂Ny[ix, iy]; ds2 = grid.Δd_s[p2n, 2]
                ∂Nz = mp.∂Nz[ix, iy]; ds3 = grid.Δd_s[p2n, 3]
                # compute solid incremental deformation gradient
                dF1 += ds1*∂Nx; dF2 += ds1*∂Ny; dF3 += ds1*∂Nz
                dF4 += ds2*∂Nx; dF5 += ds2*∂Ny; dF6 += ds2*∂Nz
                dF7 += ds3*∂Nx; dF8 += ds3*∂Ny; dF9 += ds3*∂Nz
            end
        end
        mp.ΔFs[ix, 1] = dF1; mp.ΔFs[ix, 2] = dF2; mp.ΔFs[ix, 3] = dF3
        mp.ΔFs[ix, 4] = dF4; mp.ΔFs[ix, 5] = dF5; mp.ΔFs[ix, 6] = dF6
        mp.ΔFs[ix, 7] = dF7; mp.ΔFs[ix, 8] = dF8; mp.ΔFs[ix, 9] = dF9
        # compute strain increment
        mp.Δϵij_s[ix, 1] = dF1
        mp.Δϵij_s[ix, 2] = dF5
        mp.Δϵij_s[ix, 3] = dF9
        mp.Δϵij_s[ix, 4] = dF2+dF4
        mp.Δϵij_s[ix, 5] = dF6+dF8
        mp.Δϵij_s[ix, 6] = dF3+dF7
        # update strain tensor
        mp.ϵij_s[ix, 1] += dF1
        mp.ϵij_s[ix, 2] += dF5
        mp.ϵij_s[ix, 3] += dF9
        mp.ϵij_s[ix, 4] += dF2+dF4
        mp.ϵij_s[ix, 5] += dF6+dF8
        mp.ϵij_s[ix, 6] += dF3+dF7
        # deformation gradient matrix
        F1 = mp.F[ix, 1]; F2 = mp.F[ix, 2]; F3 = mp.F[ix, 3]
        F4 = mp.F[ix, 4]; F5 = mp.F[ix, 5]; F6 = mp.F[ix, 6]
        F7 = mp.F[ix, 7]; F8 = mp.F[ix, 8]; F9 = mp.F[ix, 9]        
        mp.F[ix, 1] = (dF1+T2(1.0))*F1+dF2*F4+dF3*F7
        mp.F[ix, 2] = (dF1+T2(1.0))*F2+dF2*F5+dF3*F8
        mp.F[ix, 3] = (dF1+T2(1.0))*F3+dF2*F6+dF3*F9
        mp.F[ix, 4] = (dF5+T2(1.0))*F4+dF4*F1+dF6*F7
        mp.F[ix, 5] = (dF5+T2(1.0))*F5+dF4*F2+dF6*F8
        mp.F[ix, 6] = (dF5+T2(1.0))*F6+dF4*F3+dF6*F9
        mp.F[ix, 7] = (dF9+T2(1.0))*F7+dF8*F4+dF7*F1
        mp.F[ix, 8] = (dF9+T2(1.0))*F8+dF8*F5+dF7*F2
        mp.F[ix, 9] = (dF9+T2(1.0))*F9+dF8*F6+dF7*F3
        # update jacobian value and particle volume
        mp.J[ix] = mp.F[ix, 1]*mp.F[ix, 5]*mp.F[ix, 9]+mp.F[ix, 2]*mp.F[ix, 6]*mp.F[ix, 7]+
                   mp.F[ix, 3]*mp.F[ix, 4]*mp.F[ix, 8]-mp.F[ix, 7]*mp.F[ix, 5]*mp.F[ix, 3]-
                   mp.F[ix, 8]*mp.F[ix, 6]*mp.F[ix, 1]-mp.F[ix, 9]*mp.F[ix, 4]*mp.F[ix, 2]
    end
end

In [48]:
function testprocedure!(args    ::MODELARGS, 
                        grid    ::GRID, 
                        mp      ::PARTICLE, 
                        pts_attr::PROPERTY,
                        bc      ::BOUNDARY,
                        ΔT      ::T2,
                        Ti      ::T2,
                                ::Val{:OS},
                                ::Val{:MUSL}) where {T2}
    Ti < args.Te ? G = args.gravity / args.Te * Ti : G = args.gravity
    dev = getBackend(Val(args.device))
    resetgridstatus_OS!(dev)(ndrange=grid.node_num, grid)
    args.device == :CPU && args.basis == :uGIMP ? 
        resetmpstatus_OS_CPU!(dev)(ndrange=mp.num, grid, mp, Val(args.basis)) :
        resetmpstatus_OS!(dev)(ndrange=mp.num, grid, mp, Val(args.basis))
    P2G_OS!(dev)(ndrange=mp.num, grid, mp, G)
    solvegrid_OS!(dev)(ndrange=grid.node_num, grid, bc, ΔT, args.ζs)
    doublemapping1_OS!(dev)(ndrange=mp.num, grid, mp, pts_attr, ΔT, args.FLIP, args.PIC)
    doublemapping2_OS!(dev)(ndrange=mp.num, grid, mp)
    doublemapping3_OS!(dev)(ndrange=grid.node_num, grid, bc, ΔT)
    testG2P_OS!(dev)(ndrange=mp.num, grid, mp)
    if args.constitutive==:hyperelastic
        hyE!(dev)(ndrange=mp.num, mp, pts_attr)
    elseif args.constitutive==:linearelastic
        liE!(dev)(ndrange=mp.num, mp, pts_attr)
    elseif args.constitutive==:druckerprager
        liE!(dev)(ndrange=mp.num, mp, pts_attr)
        if Ti≥args.Te
            dpP!(dev)(ndrange=mp.num, mp, pts_attr)
        end
    elseif args.constitutive==:mohrcoulomb
        liE!(dev)(ndrange=mp.num, mp, pts_attr)
        if Ti≥args.Te
            mcP!(dev)(ndrange=mp.num, mp, pts_attr)
        end
    end
    if args.MVL == true
        vollock1_OS!(dev)(ndrange=mp.num, grid, mp)
        vollock2_OS!(dev)(ndrange=mp.num, grid, mp)
    end
    return nothing
end

testprocedure! (generic function with 1 method)

In [49]:
materialpointsolver!(args, grid, mp, pts_attr, bc, workflow=testprocedure!)

┌ Info: 2024_wish_card [3D/CUDA]
│ ────────────────┬─────────────┬─────────────────
│ ΔT  : 1.39e-04s │ PIC :  0.00 │ scheme   : MUSL
│ Ttol: 1.80e+01s │ FLIP:  1.00 │ coupling : OS
│ pts : 1.00e+06  │ ζs  :  0.00 │ animation: true
│ nds : 2.71e+06  │ ζw  :  0.00 │ precision: FP64
│ MVL :     true  │ HDF5:  true │ material : D-P
└ ────────────────┴─────────────┴─────────────────


[▲ I/O: host [≈ 3.2 GiB] → device 0 [:CUDA]


[ Info: solving 100% ◼◼◼◼◼◼◼◼◼◼◼◼  Time: 1:26:49


[▼ I/O: device 0 [CUDA] → host
[• I/O: free device 0 memory


┌ Info: performance
│ ─────────────────────
│ wtime: 01:26:50
│ iters: 1.30e+05
│ speed: 2.49e+01  it/s
│ MTeff: 1.89e+02 GiB/s
└ ─────────────────────
[ Info: ani_vtu 100% ◼◼◼◼◼◼◼◼◼◼◼◼  Time: 0:02:15


# Post processing

In [1]:
video_path = "https://raw.githubusercontent.com/LandslideSIM/.github/main/assets/MaterialPointSolver.jl/granular_render.mp4"

IJulia.display("text/html", """
<video width="640" controls>
  <source src="$video_path" type="video/mp4">
  Your browser does not support the video tag.
</video>
""")

Your browser does not support the video tag.

In [2]:
video_path = "https://raw.githubusercontent.com/LandslideSIM/.github/main/assets/MaterialPointSolver.jl/epII.mp4"

IJulia.display("text/html", """
<video width="640" controls>
  <source src="$video_path" type="video/mp4">
  Your browser does not support the video tag.
</video>
""")

Your browser does not support the video tag.